# DBiT batch RNA+ATAC panel builder (global genes across all samples)

This notebook discovers all `*_RNA.h5ad` / `*_ATAC.h5ad` pairs in a folder and builds one KaroSpace-style panel per sample using a **shared global gene list**:
- top 100 RNA genes ranked across all samples
- same 100 genes from ATAC (renamed with `ATAC_` prefix)
- total 200 features per sample with identical `var_names` across outputs
- `sample_id` and `spot_id` added to `obs`
- optional combined multi-sample output

In [17]:
from pathlib import Path
import gc

import numpy as np
import pandas as pd
import anndata as ad
from scipy import sparse

BASE_DIR = Path('/Volumes/processing2/KaroSpaceDataWrangle/dbit-data/DBiT_Di')
OUT_DIR = BASE_DIR / 'karospace_panels'

N_GENES = 200
WRITE_PER_SAMPLE = True   # Set True to write one output file per sample
WRITE_COMBINED = True     # Set True to also write one combined multi-sample file

assert BASE_DIR.exists(), f'Missing input directory: {BASE_DIR}'

In [10]:
pairs = []
missing_atac = []
for rna_path in sorted(BASE_DIR.glob('*_RNA.h5ad')):
    sample_id = rna_path.name.replace('_RNA.h5ad', '')
    atac_path = BASE_DIR / f'{sample_id}_ATAC.h5ad'
    if atac_path.exists():
        pairs.append((sample_id, rna_path, atac_path))
    else:
        missing_atac.append(sample_id)

if not pairs:
    raise ValueError(f'No RNA/ATAC pairs found in {BASE_DIR}')

print(f'Found {len(pairs)} RNA/ATAC sample pairs')
if missing_atac:
    print('Missing ATAC pairs for:', missing_atac)

pd.DataFrame(
    [{'sample_id': s, 'rna_path': str(r), 'atac_path': str(a)} for s, r, a in pairs]
).sort_values('sample_id').reset_index(drop=True)

Found 6 RNA/ATAC sample pairs


,sample_id,rna_path,atac_path
0,LPC10_S1,/Volumes/processing2/KaroSpaceDataWrangle/dbit...,/Volumes/processing2/KaroSpaceDataWrangle/dbit...
1,LPC10_S2,/Volumes/processing2/KaroSpaceDataWrangle/dbit...,/Volumes/processing2/KaroSpaceDataWrangle/dbit...
2,LPC21_S1,/Volumes/processing2/KaroSpaceDataWrangle/dbit...,/Volumes/processing2/KaroSpaceDataWrangle/dbit...
3,LPC21_S2,/Volumes/processing2/KaroSpaceDataWrangle/dbit...,/Volumes/processing2/KaroSpaceDataWrangle/dbit...
4,LPC5_S1,/Volumes/processing2/KaroSpaceDataWrangle/dbit...,/Volumes/processing2/KaroSpaceDataWrangle/dbit...
5,LPC5_S2,/Volumes/processing2/KaroSpaceDataWrangle/dbit...,/Volumes/processing2/KaroSpaceDataWrangle/dbit...


In [11]:
def _common_var_names(paths):
    common = None
    for path in paths:
        adata_b = ad.read_h5ad(path, backed='r')
        try:
            names = set(adata_b.var_names.astype(str))
        finally:
            if getattr(adata_b, 'isbacked', False):
                adata_b.file.close()
        common = names if common is None else (common & names)
    return common if common is not None else set()


def compute_global_gene_list(pairs, n_genes=100):
    rna_paths = [rna_path for _, rna_path, _ in pairs]
    atac_paths = [atac_path for _, _, atac_path in pairs]

    common_rna = _common_var_names(rna_paths)
    common_atac = _common_var_names(atac_paths)
    eligible = sorted(common_rna & common_atac)

    if len(eligible) < n_genes:
        raise ValueError(
            f'Only {len(eligible)} genes are shared across all RNA and ATAC files, requested {n_genes}.'
        )

    agg = pd.DataFrame(index=pd.Index(eligible, name='gene'))
    score_cols = []
    hvg_cols = []

    for sample_id, rna_path, _ in pairs:
        rna_b = ad.read_h5ad(rna_path, backed='r')
        try:
            var = rna_b.var.loc[eligible].copy()
        finally:
            if getattr(rna_b, 'isbacked', False):
                rna_b.file.close()

        if 'vst.variance.standardized' in var.columns:
            score = pd.to_numeric(var['vst.variance.standardized'], errors='coerce')
        elif 'highly_variable_rank' in var.columns:
            rank = pd.to_numeric(var['highly_variable_rank'], errors='coerce')
            score = -rank
        else:
            score = pd.Series(0.0, index=var.index)

        if 'vst.variable' in var.columns:
            hvg = var['vst.variable'].fillna(False).astype(bool)
        elif 'highly_variable' in var.columns:
            hvg = var['highly_variable'].fillna(False).astype(bool)
        else:
            hvg = pd.Series(True, index=var.index)

        score_col = f'{sample_id}__score'
        hvg_col = f'{sample_id}__hvg'
        agg[score_col] = score.reindex(agg.index)
        agg[hvg_col] = hvg.reindex(agg.index).fillna(False).astype(int)
        score_cols.append(score_col)
        hvg_cols.append(hvg_col)

    agg['hvg_count'] = agg[hvg_cols].sum(axis=1)
    agg['mean_score'] = agg[score_cols].mean(axis=1, skipna=True)
    agg['missing_scores'] = agg[score_cols].isna().sum(axis=1)

    ranked = agg.sort_values(
        ['hvg_count', 'mean_score', 'missing_scores'],
        ascending=[False, False, True],
    )

    selected_genes = ranked.index[:n_genes].astype(str).tolist()
    if len(selected_genes) < n_genes:
        raise ValueError(f'Unable to select {n_genes} genes globally.')

    return selected_genes, ranked.reset_index()


def build_panel_for_sample(sample_id: str, rna_path: Path, atac_path: Path, selected_genes):
    selected_genes = list(map(str, selected_genes))
    rna_b = ad.read_h5ad(rna_path, backed='r')
    atac_b = ad.read_h5ad(atac_path, backed='r')

    try:
        if rna_b.obs_names.equals(atac_b.obs_names):
            obs_names = rna_b.obs_names
        else:
            obs_names = rna_b.obs_names.intersection(atac_b.obs_names)
            if len(obs_names) == 0:
                raise ValueError(f'{sample_id}: RNA and ATAC have no shared observations')

        missing_rna = [g for g in selected_genes if g not in set(rna_b.var_names.astype(str))]
        missing_atac = [g for g in selected_genes if g not in set(atac_b.var_names.astype(str))]
        if missing_rna:
            raise ValueError(f'{sample_id}: missing selected genes in RNA: {missing_rna[:10]}')
        if missing_atac:
            raise ValueError(f'{sample_id}: missing selected genes in ATAC: {missing_atac[:10]}')

        rna_sub = rna_b[obs_names, selected_genes].to_memory()
        atac_sub = atac_b[obs_names, selected_genes].to_memory()

        if 'counts' not in rna_sub.layers:
            rna_sub.layers['counts'] = rna_sub.X.copy()
        if 'counts' not in atac_sub.layers:
            atac_sub.layers['counts'] = atac_sub.X.copy()

        rna_sub.var = rna_sub.var.copy()
        rna_sub.var['source_modality'] = 'RNA'
        rna_sub.var['paired_gene'] = rna_sub.var_names.astype(str)
        rna_sub.var['highly_variable'] = True

        atac_sub.var = atac_sub.var.copy()
        atac_sub.var['source_modality'] = 'ATAC'
        atac_sub.var['paired_gene'] = atac_sub.var_names.astype(str)
        atac_sub.var['highly_variable'] = True
        atac_sub.var_names = [f'ATAC_{g}' for g in atac_sub.var_names.astype(str)]

        obs = rna_sub.obs.copy()
        obs['sample_id'] = sample_id
        obs['spot_id'] = obs.index.astype(str)
        obs.index = [f'{sample_id}__{idx}' for idx in obs.index.astype(str)]

        X_combined = sparse.hstack([rna_sub.X, atac_sub.X], format='csr')
        var_combined = pd.concat([rna_sub.var, atac_sub.var], axis=0)

        panel = ad.AnnData(X=X_combined, obs=obs, var=var_combined)
        panel.layers['counts'] = panel.X.copy()

        if {'x', 'y'}.issubset(panel.obs.columns):
            panel.obsm['spatial'] = panel.obs[['x', 'y']].to_numpy(dtype=np.float32)

        panel.uns['panel_info'] = {
            'sample_id': sample_id,
            'source_rna': str(rna_path),
            'source_atac': str(atac_path),
            'n_rna_hvg': len(selected_genes),
            'n_atac_matched': len(selected_genes),
            'atac_feature_prefix': 'ATAC_',
            'gene_selection': 'global_across_samples',
        }

        return panel
    finally:
        if getattr(rna_b, 'isbacked', False):
            rna_b.file.close()
        if getattr(atac_b, 'isbacked', False):
            atac_b.file.close()

In [12]:
selected_genes, ranked_genes = compute_global_gene_list(pairs, n_genes=N_GENES)
print(f'Using {len(selected_genes)} global genes for all samples.')
print('Top 10 genes:', selected_genes[:10])
ranked_genes[['gene', 'hvg_count', 'mean_score']].head(20)

Using 200 global genes for all samples.
Top 10 genes: ['Rgs9', 'Rarb', 'Otx2os1', 'Tmem72', 'Gm10754', 'Sh3rf2', 'Drd2', 'Kl', 'Gmnc', 'Rbm47']


,gene,hvg_count,mean_score
0,Rgs9,6,20.146402
1,Rarb,6,18.718756
2,Otx2os1,6,17.780522
3,Tmem72,6,16.174874
4,Gm10754,6,16.070239
5,Sh3rf2,6,15.914391
6,Drd2,6,15.881085
7,Kl,6,15.047480
8,Gmnc,6,14.192311
9,Rbm47,6,13.827479


In [ ]:
if WRITE_PER_SAMPLE:
    OUT_DIR.mkdir(parents=True, exist_ok=True)

summary = []
written_files = []
reference_var_names = None

for sample_id, rna_path, atac_path in pairs:
    print(f'Processing {sample_id} ...')
    panel = build_panel_for_sample(sample_id, rna_path, atac_path, selected_genes)

    if reference_var_names is None:
        reference_var_names = panel.var_names.astype(str).tolist()
    else:
        current_var_names = panel.var_names.astype(str).tolist()
        if current_var_names != reference_var_names:
            raise ValueError(f'{sample_id}: var_names differ from the first sample output.')

    out_path = OUT_DIR / f'{sample_id}_RNA{N_GENES}_plus_ATAC{N_GENES}_karospace.h5ad'
    if WRITE_PER_SAMPLE:
        panel.write_h5ad(out_path)
        written_files.append(out_path)

    summary.append(
        {
            'sample_id': sample_id,
            'n_obs': panel.n_obs,
            'n_vars': panel.n_vars,
            'n_selected_genes_global': len(selected_genes),
            'output_path': str(out_path),
        }
    )

    del panel
    gc.collect()

summary_df = pd.DataFrame(summary).sort_values('sample_id').reset_index(drop=True)
summary_df

Processing LPC10_S1 ...
Processing LPC10_S2 ...
Processing LPC21_S1 ...
Processing LPC21_S2 ...
Processing LPC5_S1 ...


In [18]:
if WRITE_COMBINED:
    if not WRITE_PER_SAMPLE or not written_files:
        raise ValueError('WRITE_COMBINED=True requires WRITE_PER_SAMPLE=True and written sample files.')

    combined_parts = [ad.read_h5ad(path) for path in written_files]
    combined = ad.concat(combined_parts, axis=0, join='outer', merge='first', fill_value=0)
    combined.obs['sample_id'] = combined.obs['sample_id'].astype(str)

    if {'x', 'y'}.issubset(combined.obs.columns):
        combined.obsm['spatial'] = combined.obs[['x', 'y']].to_numpy(dtype=np.float32)

    combined.uns['panel_info'] = {
        'mode': 'batch_combined',
        'n_samples': len(written_files),
        'n_rna_hvg_per_sample': N_GENES,
        'n_atac_matched_per_sample': N_GENES,
        'source_dir': str(BASE_DIR),
        'gene_selection': 'global_across_samples',
    }

    combined_out = OUT_DIR / f'DBiT_all_samples_RNA{N_GENES}_plus_ATAC{N_GENES}_karospace.h5ad'
    combined.write_h5ad(combined_out)
    print(f'Wrote combined file: {combined_out}')
    print('Combined shape:', combined.shape)
else:
    print('WRITE_COMBINED is False. Set to True to write a combined multi-sample file.')

ValueError: WRITE_COMBINED=True requires WRITE_PER_SAMPLE=True and written sample files.